# difference of dealing with featurs in Darts

Darts class deal with features by another paarameter like **`past_covariates`**. Therefore, when using Darts library, we need separates additional features from **X** dataset.

In this notebook, I show how to use additional features in Darts and comapare darts and pytorch.

In [ ]:
!pip install darts

## Darts `past_covariates`

In [ ]:
from darts.models import TCNModel
from darts.datasets import AirPassengersDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts import TimeSeries
import matplotlib.pyplot as plt

# Load the target time series (e.g., monthly air passengers data)
series = AirPassengersDataset().load()

# Generate a past covariate (e.g., month of the year as a cyclic feature)
covariates = datetime_attribute_timeseries(
    series.time_index, attribute="month", cyclic=True, one_hot=False
)

# Create the TCN model
model = TCNModel(
    input_chunk_length=24,
    output_chunk_length=12,
    kernel_size=3,
    num_filters=16,
    num_layers=3,
    dropout=0.2,
    weight_norm=True,
    random_state=42
)

# Split the data into training and validation sets
train_series = series[:-36]
val_series = series[-36:]

# Ensure covariates cover the required range for training and prediction
train_covariates = covariates[:len(train_series)]
val_covariates = covariates[-(len(val_series) + model.output_chunk_length):]

# Fit the model with the past covariates
model.fit(train_series, past_covariates=train_covariates)

# Make predictions
pred_series = model.predict(n=12, series=train_series, past_covariates=covariates)

# Plot the results
series.plot(label='True Series')
pred_series.plot(label='Predictions', linestyle='dashed')
plt.legend()
plt.show()


## Pytorch codes to deal with additional features

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the target time series (e.g., monthly air passengers data)
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
data = pd.read_csv(url, usecols=[1])
series = data.values.astype(float).reshape(-1)

# Generate a past covariate (e.g., month of the year as a cyclic feature)
dates = pd.date_range(start='1949-01', periods=len(series), freq='M')
covariates = pd.get_dummies(dates.month, drop_first=True).values

# Normalize the series and covariates
scaler = StandardScaler()
series_scaled = scaler.fit_transform(series.reshape(-1, 1)).reshape(-1)
covariates_scaled = pd.get_dummies(dates.month, drop_first=True).values

# Create sequences of input data for training
def create_sequences(data, covariates, input_length, output_length):
    X, y = [], []
    for i in range(len(data) - input_length - output_length):
        combined_features = np.hstack([data[i:i+input_length].reshape(-1, 1), covariates[i:i+input_length]])
        X.append(combined_features)
        y.append(data[i+input_length:i+input_length+output_length])
    return np.array(X), np.array(y)

input_chunk_length = 24
output_chunk_length = 12
X, y = create_sequences(series_scaled, covariates_scaled, input_chunk_length, output_chunk_length)

# Debug prints to check shapes
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split the data into training and validation sets
train_size = len(X) - 36
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

# Debug prints to check shapes after split
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

# Create DataLoader for training
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                            torch.tensor(y_val, dtype=torch.float32))
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Check the shapes after DataLoader creation
for batch_x, batch_y in train_loader:
    print(f"batch_x shape: {batch_x.shape}")
    print(f"batch_y shape: {batch_y.shape}")
    break  # Only check the first batch

# Define the TCN model
class ResidualBlock(nn.Module):
    def __init__(self, num_filters, kernel_size, dilation_base, dropout, weight_norm, nr_blocks_below, num_layers, input_size, target_size):
        super(ResidualBlock, self).__init__()
        self.dilation_base = dilation_base
        self.kernel_size = kernel_size
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.num_layers = num_layers
        self.nr_blocks_below = nr_blocks_below

        input_dim = input_size if nr_blocks_below == 0 else num_filters
        output_dim = target_size if nr_blocks_below == num_layers - 1 else num_filters
        self.conv1 = nn.Conv1d(input_dim, num_filters, kernel_size, dilation=(dilation_base**nr_blocks_below))
        self.conv2 = nn.Conv1d(num_filters, output_dim, kernel_size, dilation=(dilation_base**nr_blocks_below))
        if weight_norm:
            self.conv1 = nn.utils.parametrizations.weight_norm(self.conv1)
            self.conv2 = nn.utils.parametrizations.weight_norm(self.conv2)
        if input_dim != output_dim:
            self.conv3 = nn.Conv1d(input_dim, output_dim, 1)

    def forward(self, x):
        residual = x
        left_padding = (self.dilation_base**self.nr_blocks_below) * (self.kernel_size - 1)
        x = F.pad(x, (left_padding, 0))
        x = self.dropout1(F.relu(self.conv1(x)))
        x = F.pad(x, (left_padding, 0))
        x = self.conv2(x)
        if self.nr_blocks_below < self.num_layers - 1:
            x = F.relu(x)
        x = self.dropout2(x)
        if self.conv1.in_channels != self.conv2.out_channels:
            residual = self.conv3(residual)
        x = x + residual
        return x

class TCNModel(nn.Module):
    def __init__(self, input_size, target_size, nr_params, kernel_size, num_filters, num_layers, dilation_base, target_length, dropout, weight_norm):
        super(TCNModel, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.res_blocks_list = []
        for i in range(num_layers):
            res_block = ResidualBlock(num_filters, kernel_size, dilation_base, dropout, weight_norm, i, num_layers, input_size, target_size * nr_params)
            self.res_blocks_list.append(res_block)
        self.res_blocks = nn.ModuleList(self.res_blocks_list)
        self.target_size = target_size
        self.nr_params = nr_params
        self.target_length = target_length

    def forward(self, x):
        x = x.transpose(1, 2)
        for res_block in self.res_blocks:
            x = res_block(x)
        x = x.transpose(1, 2)
        return x

# Initialize the model
input_size = X_train.shape[2]
target_size = 1
nr_params = 1
model = TCNModel(input_size, target_size, nr_params, kernel_size=3, num_filters=16, num_layers=3, dilation_base=2, target_length=output_chunk_length, dropout=0.2, weight_norm=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_x)
        output = output[:, -output_chunk_length:, :]  # Select the last output_chunk_length points
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            output = model(batch_x)
            output = output[:, -output_chunk_length:, :]  # Select the last output_chunk_length points
            loss = criterion(output.squeeze(), batch_y)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Make predictions
model.eval()
with torch.no_grad():
    X_test = torch.tensor(X_val, dtype=torch.float32)
    pred_series = model(X_test)
    pred_series = pred_series[:, -output_chunk_length:, :].squeeze().numpy()  # Select the last output_chunk_length points

# Rescale predictions
pred_series_rescaled = scaler.inverse_transform(pred_series.reshape(-1, 1)).reshape(pred_series.shape)

# Extract the last prediction from each sequence
last_predictions = pred_series_rescaled[:, -1]

# Plot the last predictions
plt.figure(figsize=(10, 6))
plt.plot(series, label='True Series')
plt.plot(range(len(series) - len(last_predictions), len(series)), last_predictions, label='Last Predictions', linestyle='dashed')
plt.legend()
plt.show()
